# Import required packages

In [1]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Flatten, Dropout, Activation
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


# Declare constants

In [2]:
IMG_DIR = '../input/utkface'
IMG_SIZE = 224
CHANNELS = 3
NUM_CLASSES = 5
BATCH_SIZE = 32
TRAIN_SIZE = 14223
TEST_SIZE = 4741
NUM_EPOCHS = 50

# Create image generators

In [3]:
data_aug_gen = ImageDataGenerator(rescale = 1./255,
                                  horizontal_flip = True)
train_gen = data_aug_gen.flow_from_directory(directory = IMG_DIR + '/Train',
                                             target_size = (IMG_SIZE, IMG_SIZE))

data_no_aug_gen = ImageDataGenerator(rescale = 1./255)
val_gen = data_no_aug_gen.flow_from_directory(directory = IMG_DIR + '/Val',
                                              target_size = (IMG_SIZE, IMG_SIZE),
                                              shuffle = False)
test_gen = data_no_aug_gen.flow_from_directory(directory = IMG_DIR + '/Test',
                                              target_size = (IMG_SIZE, IMG_SIZE),
                                              shuffle = False)

Found 14223 images belonging to 5 classes.
Found 4741 images belonging to 5 classes.
Found 4741 images belonging to 5 classes.


# Specify model

In [4]:
VGGFace = Sequential()

VGGFace.add(ZeroPadding2D(input_shape = (IMG_SIZE, IMG_SIZE, CHANNELS)))
VGGFace.add(Conv2D(64, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(64, (3, 3), activation = 'relu'))
VGGFace.add(MaxPooling2D((2, 2), strides = 2))

VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(128, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(128, (3, 3), activation = 'relu'))
VGGFace.add(MaxPooling2D((2, 2), strides = 2))

VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(256, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(256, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(256, (3, 3), activation = 'relu'))
VGGFace.add(MaxPooling2D((2, 2), strides = 2))

VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(512, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(512, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(512, (3, 3), activation = 'relu'))
VGGFace.add(MaxPooling2D((2, 2), strides = 2))

VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(512, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(512, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(512, (3, 3), activation = 'relu'))
VGGFace.add(MaxPooling2D((2, 2), strides = 2))

VGGFace.add(Conv2D(4096, (7, 7), activation = 'relu'))
VGGFace.add(Dropout(0.5))

VGGFace.add(Conv2D(4096, (1, 1), activation = 'relu'))
VGGFace.add(Dropout(0.5))

VGGFace.add(Conv2D(2622, (1, 1)))
VGGFace.add(Flatten())
VGGFace.add(Activation('softmax'))

VGGFace.load_weights('../input/vgg_face_weights.h5')

VGGFace.pop()
VGGFace.pop()
VGGFace.pop()

model = Sequential()

model.add(VGGFace)
model.add(Conv2D(NUM_CLASSES, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

model.layers[0].trainable = False

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 1, 1, 4096)        134260544 
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 1, 1, 5)           20485     
_________________________________________________________________
flatten_2 (Flatten)          (None, 5)                 0         
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
Total params: 134,281,029
Trainable params: 20,485
Non-trainable params: 134,260,544
_________________________________________________________________


# Compile model and fit to training set

In [5]:
model.compile(optimizer = 'nadam', loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])
model.fit_generator(train_gen, steps_per_epoch = TRAIN_SIZE // BATCH_SIZE + 1, epochs = NUM_EPOCHS, validation_data = val_gen, validation_steps = TEST_SIZE // BATCH_SIZE + 1)

Epoch 1/50
445/445 [==============================] - 246s 552ms/step - loss: 0.9717 - acc: 0.6978 - val_loss: 0.7641 - val_acc: 0.7951
Epoch 2/50
445/445 [==============================] - 239s 537ms/step - loss: 0.6574 - acc: 0.8050 - val_loss: 0.6397 - val_acc: 0.8094
Epoch 3/50
445/445 [==============================] - 239s 537ms/step - loss: 0.5930 - acc: 0.8133 - val_loss: 0.5933 - val_acc: 0.8140
Epoch 4/50
445/445 [==============================] - 239s 536ms/step - loss: 0.5605 - acc: 0.8176 - val_loss: 0.5681 - val_acc: 0.8216
Epoch 5/50
445/445 [==============================] - 239s 537ms/step - loss: 0.5468 - acc: 0.8177 - val_loss: 0.5540 - val_acc: 0.8244
Epoch 6/50
445/445 [==============================] - 242s 543ms/step - loss: 0.5279 - acc: 0.8237 - val_loss: 0.5425 - val_acc: 0.8258
Epoch 7/50
445/445 [==============================] - 239s 538ms/step - loss: 0.5181 - acc: 0.8271 - val_loss: 0.5356 - val_acc: 0.8288
Epoch 8/50
445/445 [============================

# Evaluate model on test set

In [6]:
loss, acc = model.evaluate_generator(test_gen, steps = TEST_SIZE // BATCH_SIZE + 1)
print("Accuracy: %.1f%%" % (100 * acc))

Accuracy: 84.5%
